In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import numpy as np
import tabula
import re
import sqlite3
from Functions.refactor_dataframe_columns import * 

C:\Users\Usuario\AppData\Local\Temp\ipykernel_19356\1440494663.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import os
allFilesinDirectory = os.listdir("../Inputs/")

In [4]:

usefulFiles = []
for k in allFilesinDirectory:
    if k.__contains__("3_1_A"):
        usefulFiles.append(k)
print(usefulFiles)

['DocEconomica_384_3_1_A_2020_12_31_00_00_00_000.pdf', 'DocEconomica_384_3_1_A_2021_12_31_00_00_00_000.pdf', 'DocEconomica_384_3_1_A_2022_12_31_00_00_00_000.pdf', 'DocEconomica_6712_3_1_A_2022_12_31_00_00_00_000.pdf', 'DocEconomica_736008_3_1_A_2021_12_31_00_00_00_000.pdf', 'DocEconomica_736008_3_1_A_2022_01_31_11_46_22_593.pdf', 'DocEconomica_736008_3_1_A_2022_12_31_00_00_00_000.pdf']


In [5]:
route = "../Inputs/"
for k in range(len(usefulFiles)):
    usefulFiles[k] = route+usefulFiles[k]
print(usefulFiles)

['../Inputs/DocEconomica_384_3_1_A_2020_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_384_3_1_A_2021_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_384_3_1_A_2022_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_6712_3_1_A_2022_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_736008_3_1_A_2021_12_31_00_00_00_000.pdf', '../Inputs/DocEconomica_736008_3_1_A_2022_01_31_11_46_22_593.pdf', '../Inputs/DocEconomica_736008_3_1_A_2022_12_31_00_00_00_000.pdf']


In [6]:
#files = (glob.glob("../Inputs/*3_1_A*"))
#files.clear()
files = [
    "../Inputs/DocEconomica_384_3_1_A_2020_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_384_3_1_A_2021_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_384_3_1_A_2022_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_6712_3_1_A_2022_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_736008_3_1_A_2021_12_31_00_00_00_000.pdf",
    "../Inputs/DocEconomica_736008_3_1_A_2022_01_31_11_46_22_593.pdf",
    "../Inputs/DocEconomica_736008_3_1_A_2022_12_31_00_00_00_000.pdf"
]

In [7]:
dataPDF = pd.DataFrame()
sqliteFile = "../Outputs/datos_ERI2.db"
fileType = "ESTADO DE RESULTADO INTEGRAL"

In [8]:
try:
    for document in usefulFiles:
        dataPDF = tabula.read_pdf(document, pages="all", multiple_tables=True, lattice=True,
                                    output_format="dataframe", encoding="latin-1")[0]
        print(dataPDF[:20])
        listaColumnas = dataPDF.columns.to_list()

        rowStartIndex = dataPDF.index[ dataPDF[listaColumnas[0]]==fileType ].to_list()
        dfHeader = dataPDF.iloc[ : rowStartIndex[0]].copy()

        dataPDF.drop([listaColumnas[1]], axis=1, inplace=True)
        dataPDF.columns = ["CUENTA","VALOR EN USD"]

        listaColumnas.clear()
        listaColumnas = dataPDF.columns.to_list()

        indexCodValor = dataPDF[ ( dataPDF[listaColumnas[0]]==listaColumnas[0] ) &
                ( dataPDF[listaColumnas[1]]=="VALOR (En USD$)" )
            ].index
        dataPDF.drop(indexCodValor, inplace=True)



        patternDelNull = r"ull"
        filterNullStrings = dataPDF[ dataPDF[listaColumnas[0]].str.contains(patternDelNull, na=False) ].index
        dataPDF.drop(filterNullStrings, inplace=True)

        filterNullStrings = dataPDF[ ( dataPDF[listaColumnas[0]].isna() ) &
                ( dataPDF[listaColumnas[1]].isna() ) ].index
        
        dataPDF.drop(filterNullStrings, inplace=True)
        dataPDF.reset_index(drop=True, inplace=True)


        rowStartIndex = dataPDF.index[ dataPDF[listaColumnas[0]]=="REPRESENTANTE(S) LEGAL(ES)" ].to_list()
        dfContador = dataPDF.iloc[ rowStartIndex[0]: ].copy()
        dfContador.columns = ["REPRESENTANTE(S) LEGAL(ES)","CONTADOR"]


        filterNullStrings = dfContador[ 
            (dfContador[dfContador.columns.to_list()[0]]=="REPRESENTANTE(S) LEGAL(ES)") |
            (dfContador[dfContador.columns.to_list()[0]]=="CONTADOR") ].index
        dfContador.drop(filterNullStrings, inplace=True)
        
        listaColumnasContador = dfContador.columns.to_list()
        dfContador.reset_index(drop=True, inplace=True)

        pattern = dfContador.loc[0,"REPRESENTANTE(S) LEGAL(ES)"].split("\r")[0]
        similar_rows = dfContador[dfContador[listaColumnasContador[0]].str.contains(pattern, case=False, na=False)]
        if(len(similar_rows) > 1):
            dfContador.drop(similar_rows.index.to_list()[0], inplace=True)
        
        dfContador.reset_index(drop=True, inplace=True)
        ColumnasContador= ["REPRESENTANTE LEGAL","IDENT. REPRESENTANTE LEGAL","CONTADOR","IDENT. CONTADOR","COD.CONTADOR"]

        datosContador = []
        if len(dfContador) >= len(ColumnasContador):
            for k in range(len(ColumnasContador)):
                datosContador.append( dfContador.loc[k, listaColumnasContador[0]] )
        else:
            for k in range(len(ColumnasContador)):
                if k in range(len(dfContador)):
                    datosContador.append( dfContador.loc[k, listaColumnasContador[0]] )
                else:
                    datosContador.append(" ")
        
        headerColumns = dfHeader.columns.to_list()
        dfHeader.drop([headerColumns[-1]], axis=1, inplace=True)
        headerColumns.pop(0)
        dfHeader.columns = headerColumns

        newFirstRow = pd.DataFrame([dfHeader.columns.to_list()], columns=dfHeader.columns)
        dataFrameHeaderContador = pd.concat([newFirstRow, dfHeader])
        dataFrameHeaderContador.reset_index(drop=True, inplace=True)

        rowStartIndex = dataPDF.index[ dataPDF[listaColumnas[0]]==fileType ].to_list()
        dataPDF = dataPDF.iloc[ rowStartIndex[0]: ]
        dataPDF.reset_index(drop=True, inplace=True)

        rowStartIndex = dataPDF.index[ dataPDF[listaColumnas[0]]=="REPRESENTANTE(S) LEGAL(ES)" ].to_list()
        dataPDF = dataPDF.iloc[ : rowStartIndex[0] ]

        filterNullStrings = dataPDF[ (dataPDF[listaColumnas[0]]==fileType) & 
                (dataPDF[listaColumnas[1]].isna()) ].index
        dataPDF.drop(filterNullStrings, inplace=True)

        filterNullStrings = dataPDF[ (dataPDF[listaColumnas[0]].isna()) & 
                (dataPDF[listaColumnas[1]].isna()) ].index
        dataPDF.drop(filterNullStrings, inplace=True)
        dataPDF.reset_index(drop=True, inplace=True)

        if len(datosContador) <= 0:
            for k in range(len(ColumnasContador)):
                datosContador.append(" ")

        dfContador = pd.DataFrame([datosContador], columns =ColumnasContador)
        transposedHeader = dataFrameHeaderContador.set_index( dataFrameHeaderContador.columns.to_list()[0]).T


        for k in range( len(dfContador.columns.to_list()) ):
            transposedHeader.insert(k, dfContador.columns.to_list()[k], dfContador.loc[0, dfContador.columns.to_list()[k]])
        
        transposedHeader.reset_index(drop=True, inplace=True)

        transposedData = dataPDF.set_index( listaColumnas[0] ).T

        transposedData.columns = [
            remove_accents_and_replace(col) for col in transposedData.columns
        ]
        datosTransposed = rename_duplicate_columns(transposedData)

        data = remove_trailing_number_in_column(datosTransposed)

        conn = sqlite3.connect(sqliteFile)
        cursor = conn.cursor()

        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='contabilidad'")
        table_exists = cursor.fetchone()

        if not table_exists:
            cursor.execute("CREATE TABLE contabilidad (id INTEGER PRIMARY KEY)")

        cursor.execute("PRAGMA table_info(contabilidad)")
        existing_columns = [column[1] for column in cursor.fetchall()]

        new_columns = [column for column in datosTransposed.columns if column not in existing_columns]

        for column in new_columns:
            cursor.execute(f"ALTER TABLE contabilidad ADD COLUMN {column} NULL")
            
        conn.commit()

        data.to_sql("contabilidad", conn, if_exists="append", index=False)
        transposedHeader.to_sql("cabecera", conn, if_exists="append", index=False)
except Exception as e:
    print(f"Error: {e}")
    print(f"Archivo: {document}")

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


                                           Unnamed: 0  \
0                                           DIRECCIÓN   
1                                          EXPEDIENTE   
2                                                 RUC   
3                                                 AÑO   
4                                          FORMULARIO   
5                        ESTADO DE RESULTADO INTEGRAL   
6                                              CUENTA   
7                  INGRESOS DE ACTIVIDADES ORDINARIAS   
8                                     VENTA DE BIENES   
9                             PRESTACION DE SERVICIOS   
10                              INGRESOS POR ASESORÍA   
11  INGRESOS POR ESTRUCTURACIÓN DE OFERTA PÚBLICA ...   
12  INGRESOS POR ESTRUCTURACIÓN DE NEGOCIOS FIDUCI...   
13                                              OTROS   
14                          CONTRATOS DE CONSTRUCCION   
15                          SUBVENCIONES DEL GOBIERNO   
16                             